<a href="https://colab.research.google.com/github/Rossel/DataQuest_Courses/blob/master/037__Working_with_Missing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COURSE 5/6: DATA CLEANING IN PYTHON: ADVANCED

# MISSION 4: Working with Missing Data

Identify and deal with missing and incorrect data.


## 1. Introduction